# Get Data

author: Murthadza bin Aznam</br>
date: 2022-09-15

---

goal: get data from FRBStats


In [13]:
import requests
import asyncio
import logging

In [27]:
def request_data(url, **kwargs) -> requests.Response:
    logging.debug(f"Awaiting data from {url}")
    return requests.get(url, **kwargs)

async def get_frb_population_data_as_csv(url:str='https://www.herta-experiment.org/frbstats/catalogue.csv', output_file='data.csv') -> str:
    result = await request_data(url)
    if result.status_code != 200:
        raise requests.RequestException(result.status_code)
    with open(output_file, 'w') as f:
        f.write(result.content.decode(result.encoding))
        logging.debug(f'Writing data to {output_file}')
    return output_file

In [29]:
await get_frb_population_data_as_csv()

'data.csv'